# Fonctions utiles

In [2]:
import numpy as np
import pandas as pd
import random
from itertools import permutations

In [3]:
def convert_attributes_into_pandas(filename):
    return pd.read_csv(filename)

In [4]:
df_attributes = convert_attributes_into_pandas("attributes_data.csv")
df_attributes

,ID,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202594,202595.jpg,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
202595,202596.jpg,-1,-1,-1,-1,-1,1,1,-1,-1,...,-1,1,1,-1,-1,-1,-1,-1,-1,1
202596,202597.jpg,-1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
202597,202598.jpg,-1,1,1,-1,-1,-1,1,-1,1,...,-1,1,-1,1,1,-1,1,-1,-1,1


In [5]:
df_test = convert_attributes_into_pandas("test_attributes.csv")
df_test

,ID,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,000094.jpg,-1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,-1,-1,-1,-1,1,1,-1,-1
94,000095.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,1,-1,-1,-1,1
95,000096.jpg,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,1,-1,-1,1
96,000097.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,1,1,-1,-1,1,-1,-1,1


In [6]:
indexes = df_attributes.columns
indexes

Index(['ID', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

Réduction matrice

In [7]:
#On souhaite pouvoir réduire la matrice selon certaines colonnes, dont les attributs correspondants seront fixés
#et ne pourront donc pas changer pendant l'algorithme génétique.
#Pour cela on va récupérer une liste d'attributs avec certaines valeurs et utiliser cette liste pour réduire la matrice

fixed_attributes = {"Male":-1,"Straight_Hair":-1,"Young":1}
fixed_attributes_test = {"Male":1}

In [8]:
def matrix_reduction(df,fixed_att):
    new_df = df
    attributes = list(fixed_att.keys())
    values = list(fixed_att.values())
    for i in range(len(attributes)):
        df.drop(df.index[df[attributes[i]]!= values[i]],inplace=True)
    return new_df.index.tolist()


In [9]:
df_attributes_list=matrix_reduction(df_attributes,fixed_attributes)
print(len(df_attributes_list))

83101


In [10]:
df_test_list = matrix_reduction(df_test,{"Male":1,"Young":-1})
print(df_test_list)
df_test

[14, 20, 29, 31, 48, 50, 51, 52, 54, 66, 67, 68, 71, 78]


,ID,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
14,000015.jpg,1,-1,-1,1,-1,-1,-1,1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,1,-1
20,000021.jpg,-1,-1,-1,-1,-1,-1,-1,1,-1,...,-1,1,1,-1,-1,-1,-1,-1,1,-1
29,000030.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,-1
31,000032.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,1,-1
48,000049.jpg,-1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
50,000051.jpg,1,-1,-1,1,1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
51,000052.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,-1
52,000053.jpg,1,-1,-1,-1,-1,1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
54,000055.jpg,-1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,1,-1,-1,-1,-1,1,-1
66,000067.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [11]:
def get_attributes_from_ID(df,photo_id):
    return pd.DataFrame(df.loc[df['ID'] == photo_id])

In [12]:
type(get_attributes_from_ID(df_test,'000018.jpg'))

pandas.core.frame.DataFrame

In [13]:
def delete_photos(df,photos_ids_list):
    new_df = df
    for i in range(len(photos_ids_list)):
        df.drop(df[df.ID == str(photos_ids_list[i])].index,inplace=True)
    return new_df

In [14]:
len(delete_photos(df_test,['000018.jpg','000066.jpg']))

14

## Algorithme génetique

In [15]:
def randomly_choose_photos(df,n):
    return random.sample(df,n)  

In [16]:
randomly_choose_photos(df_attributes_list,9)

[85999, 78940, 132451, 35800, 68604, 185844, 117437, 152265, 157510]

In [17]:
print(len(df_test_list))
randomly_choose_photos(df_test_list,6)

14


[48, 50, 78, 29, 71, 66]

In [18]:
encoded_imgs_test = np.load('encoded_imgs_test.npy')

In [19]:
def crossover_pixels(parents,pc):
    index_parents = list(range(0,len(parents)))
    p_combinations = list(permutations(index_parents,2))
    offsprings = []
    for i in range(len(p_combinations)):
        parent_1 = (parents[p_combinations[i][0]]).tolist()
        parent_2 = (parents[p_combinations[i][1]]).tolist()
        k = int(parents[0].size*pc)  #proportion pc from parent1
        offsprings.append(random.sample(parent_1,k)+ random.sample(parent_2,parents[0].size-k))
    return offsprings

In [20]:
def crossover_attributes(parent1,parent2,pc):
    list1 = parent1.columns.tolist()
    list1.remove('ID')
    #split of attributes list into two based on proportions
    k = int(len(list1)*pc)   #proportion pc from parent1
    l1 = random.sample(list1,k)
    l2 = []
    for attribute in list1 : 
        if(attribute not in l1):
            l2.append(attribute)
    attributes = l1 + l2 
    #getting each attribute respective value
    values = []
    for i in range(len(l1)):
        values.append(int(parent1.loc[:,l1[i]]))
    for j in range(len(l2)):
        values.append(int(parent1.loc[:,l2[j]]))
    
    #creating offspring as dataframe
    return pd.DataFrame(values,attributes).transpose()

In [21]:
offsprings = crossover_pixels(encoded_imgs_test,0.3)
offsprings[0][:10]

[0.0,
 0.0,
 1.9156643152236938,
 0.0,
 0.0,
 3.3253679275512695,
 0.0,
 0.0,
 2.810408115386963,
 1.4370893239974976]

In [22]:
offsprings2 = crossover_pixels(encoded_imgs_test,0.5)
offsprings2[0][5:10]

[0.0, 0.0, 1.6624845266342163, 0.718708336353302, 0.0]

In [23]:
def mutation_pixels(parents,pm):
    muted_parents = parents
    for j in range(len(parents)): 
        for i in range(parents[0].size) :
            r = random.random()
            if r < pm :
                muted_parents[j][i] = random.randint(0, 255)
    return muted_parents

In [24]:
print(encoded_imgs_test[0][:10])
random.seed(4)
new_parents = mutation_pixels(encoded_imgs_test,0.5)
print(new_parents[0][:10])

encoded_imgs_test.shape == new_parents.shape


[0.7864899 0.        0.        2.810408  0.        0.        2.4272146
 0.        0.        0.       ]
[ 52.   0.  46. 205.   0.  30. 184.  88.   0.  13.]


True

In [25]:
def mutation_attributes(offspring,pm):
    muted_offspring = offspring
    for i in range(offspring.shape[1]) :
        r = random.random()
        if r < pm :
            muted_offspring.iloc[0,i] = offspring.iloc[0,i] * (-1)
    return muted_offspring

In [26]:
encoded_imgs_test_2 = np.load('encoded_imgs_2.npy')

In [27]:
new_parents_2 = mutation_pixels(encoded_imgs_test_2,0.5)
new_parents_2[0:4]

array([[0.0000000e+00, 1.5500000e+02, 2.0900000e+02, 4.2484757e-02,
        4.7000000e+01, 1.8647681e-01, 0.0000000e+00, 2.3500000e+02,
        0.0000000e+00, 1.9700000e+02],
       [2.3400000e+02, 2.2000000e+01, 4.9000000e+01, 6.6408210e-02,
        0.0000000e+00, 1.9314201e-01, 0.0000000e+00, 1.2700000e+02,
        2.2000000e+01, 1.7300000e+02],
       [0.0000000e+00, 4.3000000e+01, 0.0000000e+00, 1.3500000e+02,
        0.0000000e+00, 1.2947442e-01, 0.0000000e+00, 0.0000000e+00,
        2.7000000e+01, 8.7000000e+01],
       [7.3000000e+01, 4.5000000e+01, 0.0000000e+00, 7.9000000e+01,
        8.2000000e+01, 1.8000000e+02, 7.9000000e+01, 3.7641350e-02,
        1.5500000e+02, 1.7700000e+02]], dtype=float32)

In [57]:
def matrix_reduction(df,fixed_att):
    new_df = df
    attributes = list(fixed_att.keys())
    values = list(fixed_att.values())
    for i in range(len(attributes)):
        df.drop(df.index[df[attributes[i]]!= values[i]],inplace=True)
    list_index = new_df.index.tolist()
    new_list_index = []
    for i in range(len(list_index)): 
        new_list_index.append(str(list_index[i]))
        if len(new_list_index[i])==1:
            new_list_index[i] = '00000' + new_list_index[i]
        if len(new_list_index[i])==2:
            new_list_index[i] = '0000' + new_list_index[i]
        if len(new_list_index[i])==3:
            new_list_index[i] = '000' + new_list_index[i]
        if len(new_list_index[i])==4:
            new_list_index[i] = '00' + new_list_index[i]
        if len(new_list_index[i])==5:
            new_list_index[i] = '0' + new_list_index[i]
            
    return new_list_index

In [59]:
fixed_attributes = {"Male":-1,"Straight_Hair":-1,"Young":1}
matrix_reduction(df_attributes,fixed_attributes)[-1]

'202598'

In [49]:
a = '10'

In [52]:
a[0]

'1'